Задание классификации изображений героев сериала симпсоны


Первым делом подключим все необходимые библиотеки

Kaggle contest: https://www.kaggle.com/competitions/journey-springfield/overview

In [ ]:
import torch
import numpy as np
from torch import optim

train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

In [ ]:
#If using colab
from google.colab import drive
drive.mount('/content/gdrive/')

!unzip -q /content/gdrive/My\ Drive/journey-springfield.zip
from pathlib import Path

In [ ]:
#Looking through the dir
!ls

In [ ]:
!nvidia-smi
import torch
torch.cuda.is_available()

In [ ]:
#There were troubles with Pillow
#Now it's working fine
#If not - uncomment the "!pip install Pillow" line 

import pickle

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F

#!pip uninstall -y Pillow
#!pip install Pillow

from collections import Counter
from matplotlib import pyplot as plt
from pathlib import Path

#from skimage import io

from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import WeightedRandomSampler
from torchvision import datasets, transforms
from tqdm.notebook import tqdm

from torch import optim

from tqdm import tqdm, tqdm_notebook

import torchvision
from torchvision import transforms
from multiprocessing.pool import ThreadPool
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from PIL import Image

import gc

import warnings
warnings.filterwarnings(action='ignore', category=DeprecationWarning)

%matplotlib inline

In [ ]:
#Working with Cuda
DEVICE = torch.device("cuda")

#Saving dirs with data
TRAIN_DIR = Path('train/simpsons_dataset')
TEST_DIR = Path('testset/testset')

#Here we do not use validation, to save images for better training
train_files = sorted(list(TRAIN_DIR.rglob('*.jpg')))
train_labels = [path.parent.name for path in train_files]
test_files = sorted(list(TEST_DIR.rglob('*.jpg')))

In [ ]:
#looking through the data
data = pd.Series(train_labels).value_counts().sort_values().to_frame("count")

print(data)

In [ ]:
#This was used to try to solve the problem with NUM_WORKERS 


#class_count = {}
#for i in range(len(data)):
#    class_count[data.iloc[i].name] = data.iloc[i].values[0]

Датасет слишком маленький => необходимо применить аугментацию

In [ ]:
BATCH_SIZE = 128
RESCALE_SIZE = 299
#The value bigger than 0 could not work on Win (try to solve it)
N_CORES = 0
DEVICE = torch.device("cuda")

In [ ]:
#Augmentations here
#Resizing the image, making the Horizontal flip, and invert
#After that we are forming tensor

augmentations = transforms.RandomChoice([
    transforms.Compose([
        transforms.Resize(size=299, max_size=300),
        transforms.CenterCrop(size=299),
        #transforms.RandomCrop(250)
    ]),
    transforms.RandomRotation(degrees=(-25, 25)),
    transforms.RandomHorizontalFlip(p=1),
    transforms.RandomInvert(p=0.3)
])
train_transforms = transforms.Compose([
    augmentations,
    transforms.Resize(size=(RESCALE_SIZE, RESCALE_SIZE)),
    transforms.Lambda(lambda x: np.array(x, dtype="float32") / 255),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) ,
])
train_dataset = datasets.ImageFolder(root=TRAIN_DIR, transform=train_transforms)

In [ ]:
#Here we creating different weights for all classes
#For classes with small number of elements weights are bigger

class_count = Counter(train_dataset.targets)
class_weights = {i: 1/c for i, c in class_count.items()}
sample_weights = [0] * len(train_dataset)
for i, (data, label) in enumerate(tqdm(train_dataset)):
    class_weight = class_weights[label]
    sample_weights[i] = class_weight

#Creating train sampler with different weights
N = max(class_count.values()) * len(class_count)  # fit to max
train_sampler = WeightedRandomSampler(sample_weights, num_samples=N, replacement=True)

#Train loader was moved to train method to save the GPU memory
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, sampler=train_sampler, num_workers=N_CORES)

Let's look on the pictures inside the dataset

In [ ]:
#Function to show images from tensor representation
def imshow(inp, title=None, plt_ax=plt, default=False):
    """Imshow для тензоров"""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt_ax.imshow(inp)
    if title is not None:
        plt_ax.set_title(title)
    plt_ax.grid(False)

In [ ]:
axes = plt.subplots(nrows=3, ncols=4, figsize=(12, 9))[1]
data, labels = next(iter(train_loader))
for ax, d, l in zip(axes.flatten(), data, labels):
    imshow(d.data.cpu(), title=train_dataset.classes[l], plt_ax=ax)

Let's build the NN

In [ ]:
#My variant of CNN for image classification

#I am using 5 layers of batchNorm to normalize data
#I am using two Sequential layers (like in AlexNet) to increase the chance for image classification

#It is not good to use more than 16 layers

class SimpleCnn(nn.Module):
  
    def __init__(self, n_classes):
        super().__init__()

        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=8, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        
        self.bn1 = nn.BatchNorm2d(8)
        
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.bn2 = nn.BatchNorm2d(16)

        self.conv3 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )

        self.bn3 = nn.BatchNorm2d(32)
        
        self.conv4 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.bn4 = nn.BatchNorm2d(64)
        
        self.conv5 =nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.bn5 = nn.BatchNorm2d(128)
        
        self.conv6 = nn.Flatten()

        self.conv8 = nn.Sequential(
            nn.Linear(in_features=3200, out_features=8192, bias=True),
            nn.ReLU(inplace=True),
        )

        self.conv9 = nn.Sequential(
            nn.Linear(in_features=8192, out_features=4096, bias=True),
            nn.ReLU(inplace=True),
        )
        self.out = nn.Linear(in_features=4096, out_features=n_classes, bias=True)
  
  
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.conv4(x)
        x = self.bn4(x)
        x = self.conv5(x)
        x = self.bn5(x)
        x = self.conv6(x)
        #x = self.conv7(x)
        x = self.conv8(x)
        x = self.conv9(x)

        x = x.view(x.size(0), -1)
        logits = self.out(x)
        return logits

Then we will introduce training course of NN

In [ ]:
def fit_epoch(model, train_loader, criterion, optimizer, scheduler):
    running_loss = 0.0
    running_corrects = 0
    processed_data = 0
  
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        preds = torch.argmax(outputs, 1)
        running_loss += (loss.item() * inputs.size(0))
        running_corrects += (torch.sum(preds == labels.data))
        processed_data += (inputs.size(0))
        
#This part was added to reduce GPU memory usage        
        del inputs
        del labels
        gc.collect()
        torch.cuda.empty_cache()

    scheduler.step()          
    train_loss = (running_loss / processed_data)
    train_acc = (running_corrects.cpu().numpy() / processed_data)
    return train_loss, train_acc

In [ ]:
def train(train_files, model, loss, optimizer, scheduler, epochs, batch_size):
   
    #Saving data to the memory here to reduce GPU memory usage
    train_loader = DataLoader(train_files, batch_size=batch_size, sampler=train_sampler, num_workers=N_CORES)

    history = []
    log_template = "\nEpoch {ep:03d} train_loss: {t_loss:0.4f} train_acc {t_acc:0.4f}"

    with tqdm(desc="epoch", total=epochs) as pbar_outer:
        for epoch in range(epochs):
            train_loss, train_acc = fit_epoch(model, train_loader, criterion, opt, scheduler)
            history.append((train_loss, train_acc))
            print("loss", train_loss)
            
            history.append((train_loss, train_acc))
            
            pbar_outer.update(1)
            tqdm.write(log_template.format(ep=epoch+1, t_loss=train_loss, t_acc=train_acc))
            
    return history

Function to predict the output

In [ ]:
def predict(model, test_loader):
    with torch.no_grad():
        logits = []
    
        for inputs in test_loader:
            inputs = inputs.to(DEVICE)
            model.eval()
            outputs = model(inputs).cpu()
            logits.append(outputs)
            
    probs = F.softmax(torch.cat(logits), dim=-1).numpy()
    return probs

In [ ]:
n_classes = len(np.unique(train_labels))
simple_cnn = SimpleCnn(n_classes).to(DEVICE)
print("we will classify :{}".format(n_classes))
print(simple_cnn)

Задаем оптимайзер, функцию потери и шедулер

In [ ]:
loss = torch.nn.CrossEntropyLoss()
opt = torch.optim.AdamW(simple_cnn.parameters(), lr=1e-4, amsgrad=True, betas=(0.9, 0.99), weight_decay=0.03)
criterion = nn.CrossEntropyLoss()
scheduler = optim.lr_scheduler.StepLR(optimizer=opt, step_size=7, gamma=0.1)

In [ ]:
#Training our model and saving loss information
history = train(model=simple_cnn, loss=loss, optimizer=opt, scheduler=scheduler, epochs=10, batch_size=BATCH_SIZE)

Строим кривые обучения моделей

In [ ]:
def build_loss_plot(loss):
    plt.figure(figsize=(15, 9))
    plt.plot(loss, label="train_loss")
    #plt.plot(val_loss, label="val_loss")
    plt.legend(loc='best')
    plt.xlabel("epochs")
    plt.ylabel("loss")
    plt.show()

In [ ]:
loss, acc = zip(*history)
build_loss_plot(loss)

Метрика f1

In [ ]:
from sklearn.metrics import f1_score

f1_score(actual_labels, y_pred, average='macro')

In [ ]:
#Creating test dataset to prove the skills of the model
class TestDataset(Dataset):
    def __init__(self, files):
        self.files = files

    def __getitem__(self, index):
        test_transforms = transforms.Compose([
            transforms.Resize(size=(RESCALE_SIZE, RESCALE_SIZE)),
            transforms.Lambda(lambda x: np.array(x, dtype="float32") / 255),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) ,
        ])
        img = Image.open(self.files[index]).convert('RGB')
        return test_transforms(img)
    
    def __len__(self):
        return len(self.files)

In [ ]:
def create_dataset_for_test(model):
    test_dataset = TestDataset(test_files)
    test_loader = DataLoader(test_dataset, shuffle=False, batch_size=BATCH_SIZE)

    #Making predictions about images
    probs = predict(model, test_loader)
    preds = [train_dataset.classes[i] for i in probs.argmax(axis=-1)]

    test_filenames = [path.name for path in test_dataset.files]
    
    return test_filenames

In [ ]:
test_filenames = create_dataset_for_test(simple_cnn)

In [ ]:
#Forming the result .csv file
pd.DataFrame({
    "Id": test_filenames,
    "Expected": preds
}).to_csv("MyCNN_aug.csv", index=False)

Score on kaggle platform: 0.99893

Картиночки для отладки работы сети

In [ ]:
import matplotlib.patches as patches
from matplotlib.font_manager import FontProperties

fig, ax = plt.subplots(nrows=3, ncols=3,figsize=(12, 12), \
                        sharey=True, sharex=True)
for fig_x in ax.flatten():
    random_characters = int(np.random.uniform(0,1000))
    im_val, label = train_dataset[random_characters]
    img_label = " ".join(map(lambda x: x.capitalize(),\
                train_dataset.label_encoder.inverse_transform([label])[0].split('_')))
    
    

    imshow(im_val.data.cpu(), \
          title=img_label,plt_ax=fig_x)
    
    actual_text = "Actual : {}".format(img_label)
            
    fig_x.add_patch(patches.Rectangle((0, 53),86,35,color='white'))
    font0 = FontProperties()
    font = font0.copy()
    font.set_family("fantasy")
    prob_pred = predict_one_sample(simple_cnn, im_val.unsqueeze(0))
    predicted_proba = np.max(prob_pred)*100
    y_pred = np.argmax(prob_pred)
    
    predicted_label = label_encoder.classes_[y_pred]
    predicted_label = predicted_label[:len(predicted_label)//2] + '\n' + predicted_label[len(predicted_label)//2:]
    predicted_text = "{} : {:.0f}%".format(predicted_label,predicted_proba)
            
    fig_x.text(1, 59, predicted_text + ' here ' , horizontalalignment='left', fontproperties=font,
                    verticalalignment='top',fontsize=8, color='black',fontweight='bold')

In [ ]:
#Using VGG16

In [ ]:
import os
from tqdm.autonotebook import tqdm, trange

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time

In [ ]:
#Extracting the VGG 16 model
model_extractor = models.vgg16(pretrained=True)

#Creating 4 unfreezed layers
layers_to_unfreeze = 4

#Freezing layers according to the statement
for param in model_extractor.features[:-layers_to_unfreeze].parameters():
    param.requires_grad = False

# num_features -- features size, going to the FC-layer (look through the model to understand)
num_features = 25088
n_classes = len(np.unique(train_labels))
#Changing fully-connected layer to ours to classificate our images
model_extractor.classifier = nn.Linear(num_features, n_classes)

if torch.cuda.is_available():
    model_extractor = model_extractor.to(DEVICE)

#Forming loss, optimizer, criterion an scheduler
loss = torch.nn.CrossEntropyLoss()
opt = torch.optim.AdamW(model_extractor.parameters(), lr=1e-4, amsgrad=True, betas=(0.9, 0.99), weight_decay=0.03)
criterion = nn.CrossEntropyLoss()
scheduler = optim.lr_scheduler.StepLR(optimizer=opt, step_size=4, gamma=0.1)

print(model_extractor)

In [ ]:
#Clearing the cache
torch.cuda.empty_cache()
gc.collect()
torch.cuda.memory_summary(device=None, abbreviated=False)

In [ ]:
#Training our model and saving the loss
history_vgg16 = train(train_dataset, model=model_extractor, loss=loss, optimizer=opt, scheduler=scheduler, 
                      epochs=20, batch_size=128)

In [ ]:
test_filenames = create_dataset_for_test(model_extractor)

In [ ]:
#Saving the predictions
pd.DataFrame({
    "Id": test_filenames,
    "Expected": preds
}).to_csv("transferLr_aug.csv", index=False)

In [ ]:
loss_vgg16, acc_vgg16 = zip(*history_vgg16)
build_loss_plot(loss_vgg16)

In [ ]:
#Saving the model
torch.save(model_extractor.state_dict(), 'VGG16.pth')

Score on kaggle platform: 0.97874

Let's try inception v_3

In [ ]:
#Extracting model 
model_inception = models.inception_v3(pretrained=True)
print(model_inception)

In [ ]:
n_classes = len(np.unique(train_labels))

#Forming two layers of the network to classificate my images
model_inception.AuxLogits.fc = nn.Linear(768, n_classes)
model_inception.fc = nn.Linear(2048, n_classes)

In [ ]:

# num_features -- features size, going to the FC-layer (look through the model to understand)
num_features = 25088

#Changing fully connected layer to my own layer
model_inception.classifier = nn.Linear(num_features, n_classes)

#Using GPU
model_inception = model_inception.to(DEVICE)


loss_fn = nn.CrossEntropyLoss()
opt = optim.AdamW(model_inception.parameters(), lr=1e-4, amsgrad=True, betas=(0.9, 0.99), weight_decay=0.03)
criterion = nn.CrossEntropyLoss()
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer=opt, step_size=3, gamma=0.1)

In [ ]:
%%time
model_inception.aux_logits = False

#Training model and saving the loss 
history_inception = train(train_dataset, model=model_inception, loss=loss_fn, optimizer=optimizer, scheduler=exp_lr_scheduler, 
                          epochs=10, batch_size=32)

In [ ]:
test_filenames = create_dataset_for_test(model_inception)

In [ ]:
pd.DataFrame({
    "Id": test_filenames,
    "Expected": preds
}).to_csv("inceptionModel_aug.csv", index=False)

In [ ]:
loss_inception, acc_inception = zip(*history_inception)

In [ ]:
build_loss_plot(loss_inception)

In [ ]:
#Saving the model
torch.save(model_inception.state_dict(), 'Inception.pth')